In [1]:
import pandas as pd
import numpy as np
import math 
import statistics

In [2]:
def load_dataset(file_name, category_col_name):
    X = pd.read_csv(file_name)
    y=X[category_col_name]
    X.drop(columns=category_col_name,axis=1,inplace=True)
    return X, y

In [3]:
def scale_fit_transform(X_train):
    X_train_scaled = X_train.copy().astype(float)
    scaling_info=pd.DataFrame()
    scaling_info['min'] = X_train_scaled.min(axis=0)
    scaling_info['max'] = X_train_scaled.max(axis=0)
    for col in X_train_scaled.columns:
        for i in range(len(X_train_scaled.index)):
            X_train_scaled[col][i]=((2*(X_train_scaled[col][i]-scaling_info["min"][col]))
                                    /(scaling_info["max"][col]-scaling_info["min"][col])-1)
    return scaling_info, X_train_scaled

In [4]:
def scale_transform_for_test(trained_scaling_info,  X_test):
    X_test_scaled = X_test.copy().astype(float)
    for col in X_test_scaled.columns:
        for i in (X_test_scaled.index):
            X_test_scaled[col][i]=((2*(X_test_scaled[col][i]-trained_scaling_info["min"][col]))/(trained_scaling_info["max"][col]-trained_scaling_info["min"][col])-1)
    return X_test_scaled

In [2]:
def calc_distance(X_test,X_train):
    df_dist = pd.DataFrame(0,columns=X_train.index,index=X_test.index).astype(float)
    
    for i in X_train.index:
        for j in X_test.index:
            for col in X_train.columns:
                temp = X_test[col][j] - X_train[col][i]
                if temp < 0:
                    temp=temp*(-1)
                df_dist[i][j] = temp + df_dist[i][j]
    return df_dist

In [6]:
def predict(X_test, X_train, y_train, k):
    y_predicted = pd.Series(index=[80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99],dtype='float64')
    dist_dataframe = calc_distance(X_test,X_train)
    neighbors={}
    
    
    for i in dist_dataframe.index:## makeing dict
        temp={i:[]}
        neighbors.update(temp)
    
    for i in dist_dataframe.index:## adding neighbors to the dict
        sortedRowIndexs = dist_dataframe.loc[i].sort_values().index
        for n in range(k):
            neighbors[i].append(sortedRowIndexs[n])
    
    for i in neighbors: # check the most common type
        count=[]
        for n in range(k):
            count.append(y_train[neighbors.get(i)[n]])
            y_predicted[i]=max(count)
       
    return y_predicted.astype(int)

In [7]:
def eval_performance(y_predicted,y_test):
    TP=FP=TN=FN=0
    for i in y_test.index:
        if y_predicted[i]==1 and y_test[i]==1:
            TP=TP+1
        elif y_predicted[i]==1 and y_test[i]==0:
            FP=FP+1
        elif y_predicted[i]==0 and y_test[i]==0:
            TN=TN+1
        elif y_predicted[i]==0 and y_test[i]==1:
            FN=FN+1
              
    evaluate_value=(FP+FN)/len(y_test)
    return evaluate_value

In [1]:
file_name = '/Users/noamrippa/Assignment2_KNN/Students-Performance_scaled_100.csv'
y_col     = 'category'
